In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pip install category-encoders --ignore-installed --no-deps
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lazytransform --ignore-installed --no-deps

In [ ]:
!pip install category-encoders --ignore-installed --no-deps

In [ ]:
from lazytransform import LazyTransformer

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv', parse_dates=['date']).drop('row_id',axis=1)
print(df.shape)
df.head()

In [ ]:
target = 'num_sold'
modeltype = 'Regression'

In [ ]:
if isinstance(target, str):
    cols = [x for x in list(df) if x not in [target]]
else:
    cols = [x for x in list(df) if x not in target]
X = df[cols]
y = df[target]
df.dtypes

In [ ]:
###  find the y values that have low samples ####
print(X.shape, y.shape)
if modeltype != 'Regression':
    low_counts = y.value_counts()[(y.value_counts()<=1).values].index
    print(len(low_counts))
    ## You need to remove those rows that have just one sample ##
    X = X[~(y.isin(low_counts))]
    y = y[~(y.isin(low_counts))]
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
if modeltype == 'Regression':
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=999)
else:
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,stratify=y, random_state=999)
print(X_train.shape, X_test.shape)

In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor
if modeltype == 'Regression':
        lgb = LGBMRegressor(n_estimators=100, n_jobs=-1, random_state=99)
else:
    lgb = LGBMClassifier(n_estimators=100, n_jobs=-1, random_state=99)

In [ ]:
### This returns a pipeline ## you need to fit it to see how well it performs on a hold out dataset
lazy = LazyTransformer(model=lgb, encoders='auto', scalers='max', 
        date_to_string=False, transform_target=False, imbalanced=False, verbose=2)

In [ ]:
lazy.fit(X_train, y_train)

In [ ]:
from sklearn import set_config
#### You can display your pipeline in a nice diagram if it is a Jupyter Notebook ##
set_config(display="diagram")

In [ ]:
predictions = lazy.predict(X_test)
predictions

In [ ]:
y_test.values

In [ ]:
from sklearn.metrics import balanced_accuracy_score, mean_squared_error, classification_report
if modeltype == 'Regression':
    print('RMSE = %0.2f' %np.sqrt(mean_squared_error(y_test.values, predictions)))
else:
    print(classification_report(y_test.values, predictions))

In [ ]:
### Now fit it on the entire Train dataset ###########
lazy.fit(X,y)

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv', parse_dates=['date']).drop('row_id',axis=1)
print(test.shape)
test.head()

In [ ]:
lazy.modelformer

In [ ]:
test[cols].head()

In [ ]:
y_pred = lazy.predict(test[cols])
y_pred

In [ ]:
subm = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')
print(subm.shape)
subm.head()

In [ ]:
subm[target] = y_pred
subm.head()

In [ ]:
subm.to_csv('submission.csv', index=False)